# Week 17: Delta Hedging Trading Strategy

## Complete Implementation with Performance Metrics

**Objectives:**
- Implement Black-Scholes option pricing and Greeks
- Build a complete delta hedging simulation framework
- Analyze hedging performance under different market conditions
- Calculate comprehensive performance metrics
- Understand hedging errors and their sources

---

## 1. Setup and Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

Libraries loaded successfully!


## 2. Black-Scholes Option Pricing Framework

### 2.1 Black-Scholes Formulas

The Black-Scholes model prices European options:

$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$
$$P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

In [2]:
class BlackScholesModel:
    """
    Complete Black-Scholes Option Pricing Model with Greeks.
    """
    
    def __init__(self, S, K, T, r, sigma, option_type='call'):
        """
        Initialize Black-Scholes model.
        
        Parameters:
        -----------
        S : float - Current stock price
        K : float - Strike price
        T : float - Time to maturity (in years)
        r : float - Risk-free interest rate
        sigma : float - Volatility
        option_type : str - 'call' or 'put'
        """
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type.lower()
        
    def _d1(self):
        """Calculate d1 parameter."""
        if self.T <= 0:
            return np.inf if self.S > self.K else -np.inf
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def _d2(self):
        """Calculate d2 parameter."""
        if self.T <= 0:
            return self._d1()
        return self._d1() - self.sigma * np.sqrt(self.T)
    
    def price(self):
        """Calculate option price."""
        if self.T <= 0:
            # At expiration
            if self.option_type == 'call':
                return max(self.S - self.K, 0)
            else:
                return max(self.K - self.S, 0)
        
        d1, d2 = self._d1(), self._d2()
        
        if self.option_type == 'call':
            return self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            return self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)
    
    def delta(self):
        """Calculate option delta (sensitivity to underlying price)."""
        if self.T <= 0:
            if self.option_type == 'call':
                return 1.0 if self.S > self.K else 0.0
            else:
                return -1.0 if self.S < self.K else 0.0
        
        d1 = self._d1()
        if self.option_type == 'call':
            return norm.cdf(d1)
        else:
            return norm.cdf(d1) - 1
    
    def gamma(self):
        """Calculate option gamma (sensitivity of delta to underlying price)."""
        if self.T <= 0:
            return 0.0
        d1 = self._d1()
        return norm.pdf(d1) / (self.S * self.sigma * np.sqrt(self.T))
    
    def theta(self):
        """Calculate option theta (time decay per day)."""
        if self.T <= 0:
            return 0.0
        d1, d2 = self._d1(), self._d2()
        
        term1 = -(self.S * norm.pdf(d1) * self.sigma) / (2 * np.sqrt(self.T))
        
        if self.option_type == 'call':
            term2 = -self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            term2 = self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2)
        
        return (term1 + term2) / 365  # Per day
    
    def vega(self):
        """Calculate option vega (sensitivity to volatility)."""
        if self.T <= 0:
            return 0.0
        d1 = self._d1()
        return self.S * norm.pdf(d1) * np.sqrt(self.T) / 100  # Per 1% change
    
    def rho(self):
        """Calculate option rho (sensitivity to interest rate)."""
        if self.T <= 0:
            return 0.0
        d2 = self._d2()
        
        if self.option_type == 'call':
            return self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2) / 100
        else:
            return -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2) / 100
    
    def all_greeks(self):
        """Return all Greeks as a dictionary."""
        return {
            'delta': self.delta(),
            'gamma': self.gamma(),
            'theta': self.theta(),
            'vega': self.vega(),
            'rho': self.rho()
        }

# Test the model
bs = BlackScholesModel(S=100, K=100, T=0.25, r=0.05, sigma=0.2, option_type='call')
print(f"Call Option Price: ${bs.price():.4f}")
print(f"\nGreeks:")
for greek, value in bs.all_greeks().items():
    print(f"  {greek.capitalize()}: {value:.6f}")

Call Option Price: $4.6150

Greeks:
  Delta: 0.569460
  Gamma: 0.039288
  Theta: -0.028696
  Vega: 0.196440
  Rho: 0.130828


## 3. Stock Price Simulation (Geometric Brownian Motion)

We simulate stock prices using GBM:

$$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Discretized:
$$S_{t+\Delta t} = S_t \exp\left((\mu - \frac{\sigma^2}{2})\Delta t + \sigma\sqrt{\Delta t}Z\right)$$

In [ ]:
class StockSimulator:
    """
    Simulate stock prices using Geometric Brownian Motion.
    """
    
    def __init__(self, S0, mu, sigma, T, n_steps, n_paths=1):
        """
        Initialize stock simulator.
        
        Parameters:
        -----------
        S0 : float - Initial stock price
        mu : float - Expected return (drift)
        sigma : float - Volatility
        T : float - Time horizon (in years)
        n_steps : int - Number of time steps
        n_paths : int - Number of simulation paths
        """
        self.S0 = S0
        self.mu = mu
        self.sigma = sigma
        self.T = T
        self.n_steps = n_steps
        self.n_paths = n_paths
        self.dt = T / n_steps
        
    def simulate(self, seed=None):
        """
        Simulate stock price paths.
        
        Returns:
        --------
        tuple: (times, prices) where prices shape is (n_paths, n_steps+1)
        """
        if seed is not None:
            np.random.seed(seed)
        
        # Generate random shocks
        Z = np.random.standard_normal((self.n_paths, self.n_steps))
        
        # Initialize price array
        S = np.zeros((self.n_paths, self.n_steps + 1))
        S[:, 0] = self.S0
        
        # Simulate paths
        for t in range(1, self.n_steps + 1):
            S[:, t] = S[:, t-1] * np.exp(
                (self.mu - 0.5 * self.sigma**2) * self.dt + 
                self.sigma * np.sqrt(self.dt) * Z[:, t-1]
            )
        
        times = np.linspace(0, self.T, self.n_steps + 1)
        return times, S
    
    def simulate_with_stochastic_vol(self, kappa=2.0, theta=0.04, xi=0.3, rho=-0.7, v0=0.04, seed=None):
        """
        Simulate using Heston stochastic volatility model.
        
        Parameters:
        -----------
        kappa : float - Mean reversion speed
        theta : float - Long-term variance
        xi : float - Volatility of variance
        rho : float - Correlation between stock and variance
        v0 : float - Initial variance
        """
        if seed is not None:
            np.random.seed(seed)
        
        S = np.zeros((self.n_paths, self.n_steps + 1))
        v = np.zeros((self.n_paths, self.n_steps + 1))
        S[:, 0] = self.S0
        v[:, 0] = v0
        
        for t in range(1, self.n_steps + 1):
            Z1 = np.random.standard_normal(self.n_paths)
            Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.standard_normal(self.n_paths)
            
            # Variance process (ensure non-negative)
            v[:, t] = np.maximum(
                v[:, t-1] + kappa * (theta - v[:, t-1]) * self.dt + 
                xi * np.sqrt(np.maximum(v[:, t-1], 0) * self.dt) * Z2,
                0
            )
            
            # Stock process
            S[:, t] = S[:, t-1] * np.exp(
                (self.mu - 0.5 * v[:, t-1]) * self.dt + 
                np.sqrt(np.maximum(v[:, t-1], 0) * self.dt) * Z1
            )
        
        times = np.linspace(0, self.T, self.n_steps + 1)
        return times, S, np.sqrt(v)  # Return realized volatility

# Test simulation
sim = StockSimulator(S0=100, mu=0.08, sigma=0.2, T=0.25, n_steps=63, n_paths=5)
times, prices = sim.simulate(seed=42)

plt.figure(figsize=(12, 5))
for i in range(5):
    plt.plot(times * 252, prices[i], label=f'Path {i+1}', alpha=0.7)
plt.xlabel('Trading Days')
plt.ylabel('Stock Price ($)')
plt.title('Simulated Stock Price Paths (GBM)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Delta Hedging Strategy Implementation

### Delta Hedging Concept

Delta hedging involves maintaining a position in the underlying asset to offset changes in option value:

- **Short Call Position**: Sell call option, buy $\Delta$ shares of stock
- **Rebalancing**: Adjust stock position as delta changes
- **Goal**: Create a risk-neutral portfolio

In [ ]:
class DeltaHedgingStrategy:
    """
    Complete Delta Hedging Strategy with detailed tracking.
    """
    
    def __init__(self, S0, K, T, r, sigma, option_type='call', 
                 n_options=1, transaction_cost=0.001):
        """
        Initialize delta hedging strategy.
        
        Parameters:
        -----------
        S0 : float - Initial stock price
        K : float - Strike price
        T : float - Time to maturity (years)
        r : float - Risk-free rate
        sigma : float - Implied volatility (for hedging)
        option_type : str - 'call' or 'put'
        n_options : int - Number of options sold (negative = short)
        transaction_cost : float - Transaction cost as % of trade value
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type
        self.n_options = n_options
        self.transaction_cost = transaction_cost
        
        # Calculate initial option price
        bs = BlackScholesModel(S0, K, T, r, sigma, option_type)
        self.initial_option_price = bs.price()
        
    def run_hedging_simulation(self, stock_prices, times, rebalance_freq=1):
        """
        Run delta hedging simulation.
        
        Parameters:
        -----------
        stock_prices : array - Simulated stock prices
        times : array - Time points (in years)
        rebalance_freq : int - Rebalance every N periods
        
        Returns:
        --------
        dict : Detailed hedging results
        """
        n_steps = len(stock_prices) - 1
        dt = times[1] - times[0]
        
        # Initialize tracking arrays
        results = {
            'times': times,
            'stock_prices': stock_prices,
            'option_values': np.zeros(n_steps + 1),
            'deltas': np.zeros(n_steps + 1),
            'gammas': np.zeros(n_steps + 1),
            'stock_position': np.zeros(n_steps + 1),
            'cash_position': np.zeros(n_steps + 1),
            'portfolio_value': np.zeros(n_steps + 1),
            'hedge_error': np.zeros(n_steps + 1),
            'transaction_costs': np.zeros(n_steps + 1),
            'cumulative_costs': np.zeros(n_steps + 1),
            'rebalances': []
        }
        
        # Initial setup
        S = stock_prices[0]
        tau = self.T - times[0]
        
        bs = BlackScholesModel(S, self.K, tau, self.r, self.sigma, self.option_type)
        initial_delta = bs.delta()
        initial_option_value = bs.price()
        
        # Short option, long delta shares
        results['option_values'][0] = initial_option_value
        results['deltas'][0] = initial_delta
        results['gammas'][0] = bs.gamma()
        results['stock_position'][0] = self.n_options * initial_delta  # Hedge position
        
        # Initial cash = option premium received - cost of hedge
        option_premium = self.n_options * initial_option_value
        stock_cost = results['stock_position'][0] * S
        initial_tc = abs(stock_cost) * self.transaction_cost
        
        results['cash_position'][0] = option_premium - stock_cost - initial_tc
        results['transaction_costs'][0] = initial_tc
        results['cumulative_costs'][0] = initial_tc
        results['portfolio_value'][0] = (
            results['cash_position'][0] + 
            results['stock_position'][0] * S - 
            self.n_options * initial_option_value
        )
        
        # Simulate hedging through time
        for t in range(1, n_steps + 1):
            S = stock_prices[t]
            tau = max(self.T - times[t], 0)
            
            # Calculate current option value and greeks
            bs = BlackScholesModel(S, self.K, tau, self.r, self.sigma, self.option_type)
            current_option_value = bs.price()
            current_delta = bs.delta()
            current_gamma = bs.gamma()
            
            results['option_values'][t] = current_option_value
            results['deltas'][t] = current_delta
            results['gammas'][t] = current_gamma
            
            # Cash grows at risk-free rate
            results['cash_position'][t] = results['cash_position'][t-1] * np.exp(self.r * dt)
            
            # Check if rebalancing
            if t % rebalance_freq == 0 or t == n_steps:
                # Rebalance to new delta
                new_stock_position = self.n_options * current_delta
                shares_traded = new_stock_position - results['stock_position'][t-1]
                
                # Transaction costs
                tc = abs(shares_traded * S) * self.transaction_cost
                
                # Update positions
                results['stock_position'][t] = new_stock_position
                results['cash_position'][t] -= shares_traded * S + tc
                results['transaction_costs'][t] = tc
                results['rebalances'].append(t)
            else:
                # No rebalancing
                results['stock_position'][t] = results['stock_position'][t-1]
                results['transaction_costs'][t] = 0
            
            results['cumulative_costs'][t] = results['cumulative_costs'][t-1] + results['transaction_costs'][t]
            
            # Portfolio value = cash + stock - option liability
            results['portfolio_value'][t] = (
                results['cash_position'][t] + 
                results['stock_position'][t] * S - 
                self.n_options * current_option_value
            )
            
            # Hedge error (deviation from zero P&L)
            results['hedge_error'][t] = results['portfolio_value'][t] - results['portfolio_value'][0] * np.exp(self.r * times[t])
        
        return results
    
    def calculate_final_pnl(self, results):
        """
        Calculate final P&L at expiration.
        """
        S_final = results['stock_prices'][-1]
        
        # Option payoff
        if self.option_type == 'call':
            option_payoff = max(S_final - self.K, 0)
        else:
            option_payoff = max(self.K - S_final, 0)
        
        # Final cash position
        final_cash = results['cash_position'][-1]
        
        # Liquidate stock position
        stock_value = results['stock_position'][-1] * S_final
        liquidation_cost = abs(stock_value) * self.transaction_cost
        
        # Option settlement (we're short)
        option_settlement = -self.n_options * option_payoff
        
        # Total P&L
        total_pnl = final_cash + stock_value - liquidation_cost + option_settlement
        
        return {
            'final_stock_price': S_final,
            'option_payoff': option_payoff,
            'option_settlement': option_settlement,
            'final_cash': final_cash,
            'stock_value': stock_value,
            'liquidation_cost': liquidation_cost,
            'total_pnl': total_pnl,
            'cumulative_transaction_costs': results['cumulative_costs'][-1] + liquidation_cost
        }

print("Delta Hedging Strategy class defined successfully!")

## 5. Single Path Hedging Analysis

In [ ]:
# Parameters
S0 = 100        # Initial stock price
K = 100         # ATM strike
T = 0.25        # 3 months to expiration
r = 0.05        # Risk-free rate
sigma = 0.20    # Volatility
n_steps = 63    # Daily rebalancing for ~3 months

# Simulate stock path
sim = StockSimulator(S0=S0, mu=0.08, sigma=sigma, T=T, n_steps=n_steps, n_paths=1)
times, prices = sim.simulate(seed=123)
stock_prices = prices[0]

# Initialize hedging strategy (short 1 call option)
strategy = DeltaHedgingStrategy(
    S0=S0, K=K, T=T, r=r, sigma=sigma,
    option_type='call', n_options=-1,
    transaction_cost=0.001
)

# Run daily hedging
results = strategy.run_hedging_simulation(stock_prices, times, rebalance_freq=1)

# Calculate final P&L
final_pnl = strategy.calculate_final_pnl(results)

print("=" * 50)
print("DELTA HEDGING SIMULATION RESULTS")
print("=" * 50)
print(f"\nInitial Option Premium Received: ${strategy.initial_option_price:.4f}")
print(f"Initial Stock Price: ${S0:.2f}")
print(f"Final Stock Price: ${final_pnl['final_stock_price']:.2f}")
print(f"\nOption Payoff at Expiration: ${final_pnl['option_payoff']:.4f}")
print(f"Option Settlement (short): ${final_pnl['option_settlement']:.4f}")
print(f"\nFinal Cash Position: ${final_pnl['final_cash']:.4f}")
print(f"Final Stock Value: ${final_pnl['stock_value']:.4f}")
print(f"\nTotal Transaction Costs: ${final_pnl['cumulative_transaction_costs']:.4f}")
print(f"\n{'='*50}")
print(f"TOTAL P&L: ${final_pnl['total_pnl']:.4f}")
print(f"{'='*50}")

In [ ]:
# Visualize single path hedging
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

trading_days = times * 252

# 1. Stock Price Path
ax1 = axes[0, 0]
ax1.plot(trading_days, results['stock_prices'], 'b-', linewidth=1.5)
ax1.axhline(y=K, color='r', linestyle='--', label=f'Strike = ${K}')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Stock Price ($)')
ax1.set_title('Stock Price Path')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Delta Evolution
ax2 = axes[0, 1]
ax2.plot(trading_days, np.abs(results['deltas']), 'g-', linewidth=1.5)
ax2.set_xlabel('Trading Days')
ax2.set_ylabel('Delta')
ax2.set_title('Option Delta Over Time')
ax2.set_ylim(0, 1.1)
ax2.grid(True, alpha=0.3)

# 3. Gamma Evolution
ax3 = axes[0, 2]
ax3.plot(trading_days, results['gammas'], 'purple', linewidth=1.5)
ax3.set_xlabel('Trading Days')
ax3.set_ylabel('Gamma')
ax3.set_title('Option Gamma Over Time')
ax3.grid(True, alpha=0.3)

# 4. Stock Position (Hedge)
ax4 = axes[1, 0]
ax4.plot(trading_days, results['stock_position'], 'orange', linewidth=1.5)
ax4.set_xlabel('Trading Days')
ax4.set_ylabel('Shares')
ax4.set_title('Stock Hedge Position')
ax4.grid(True, alpha=0.3)

# 5. Portfolio Value
ax5 = axes[1, 1]
ax5.plot(trading_days, results['portfolio_value'], 'navy', linewidth=1.5)
ax5.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax5.set_xlabel('Trading Days')
ax5.set_ylabel('Portfolio Value ($)')
ax5.set_title('Hedged Portfolio Value')
ax5.grid(True, alpha=0.3)

# 6. Cumulative Transaction Costs
ax6 = axes[1, 2]
ax6.plot(trading_days, results['cumulative_costs'], 'red', linewidth=1.5)
ax6.set_xlabel('Trading Days')
ax6.set_ylabel('Cumulative Costs ($)')
ax6.set_title('Transaction Costs')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Monte Carlo Hedging Performance Analysis

In [ ]:
def run_monte_carlo_hedging(n_simulations, S0, K, T, r, sigma, mu, n_steps, 
                            rebalance_freq=1, transaction_cost=0.001):
    """
    Run Monte Carlo simulation of delta hedging strategy.
    """
    pnl_results = []
    hedge_errors = []
    total_costs = []
    
    simulator = StockSimulator(S0=S0, mu=mu, sigma=sigma, T=T, n_steps=n_steps, n_paths=n_simulations)
    times, all_prices = simulator.simulate(seed=42)
    
    for i in range(n_simulations):
        strategy = DeltaHedgingStrategy(
            S0=S0, K=K, T=T, r=r, sigma=sigma,
            option_type='call', n_options=-1,
            transaction_cost=transaction_cost
        )
        
        results = strategy.run_hedging_simulation(all_prices[i], times, rebalance_freq)
        final_pnl = strategy.calculate_final_pnl(results)
        
        pnl_results.append(final_pnl['total_pnl'])
        hedge_errors.append(results['hedge_error'][-1])
        total_costs.append(final_pnl['cumulative_transaction_costs'])
    
    return np.array(pnl_results), np.array(hedge_errors), np.array(total_costs)

# Run Monte Carlo simulation
n_simulations = 1000

print(f"Running {n_simulations} Monte Carlo simulations...")
pnl_daily, errors_daily, costs_daily = run_monte_carlo_hedging(
    n_simulations=n_simulations,
    S0=100, K=100, T=0.25, r=0.05, sigma=0.20, mu=0.08,
    n_steps=63, rebalance_freq=1, transaction_cost=0.001
)
print("Simulation complete!")

# Statistics
print(f"\n{'='*60}")
print("MONTE CARLO HEDGING PERFORMANCE (Daily Rebalancing)")
print(f"{'='*60}")
print(f"\nP&L Statistics:")
print(f"  Mean P&L: ${np.mean(pnl_daily):.4f}")
print(f"  Std Dev P&L: ${np.std(pnl_daily):.4f}")
print(f"  Min P&L: ${np.min(pnl_daily):.4f}")
print(f"  Max P&L: ${np.max(pnl_daily):.4f}")
print(f"  5th Percentile: ${np.percentile(pnl_daily, 5):.4f}")
print(f"  95th Percentile: ${np.percentile(pnl_daily, 95):.4f}")
print(f"\nAvg Transaction Costs: ${np.mean(costs_daily):.4f}")

In [ ]:
# Visualize Monte Carlo results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. P&L Distribution
ax1 = axes[0]
ax1.hist(pnl_daily, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(x=np.mean(pnl_daily), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(pnl_daily):.4f}')
ax1.axvline(x=0, color='green', linestyle='-', linewidth=2, label='Break-even')
ax1.set_xlabel('P&L ($)')
ax1.set_ylabel('Frequency')
ax1.set_title('Hedging P&L Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Hedge Error Distribution
ax2 = axes[1]
ax2.hist(errors_daily, bins=50, edgecolor='black', alpha=0.7, color='coral')
ax2.axvline(x=np.mean(errors_daily), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(errors_daily):.4f}')
ax2.set_xlabel('Hedge Error ($)')
ax2.set_ylabel('Frequency')
ax2.set_title('Hedge Error Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Transaction Costs Distribution
ax3 = axes[2]
ax3.hist(costs_daily, bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
ax3.axvline(x=np.mean(costs_daily), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(costs_daily):.4f}')
ax3.set_xlabel('Transaction Costs ($)')
ax3.set_ylabel('Frequency')
ax3.set_title('Transaction Costs Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Rebalancing Frequency Analysis

In [ ]:
# Compare different rebalancing frequencies
rebalance_frequencies = [1, 2, 5, 10, 21]  # Daily, every 2 days, weekly, bi-weekly, monthly
frequency_names = ['Daily', 'Every 2 Days', 'Weekly', 'Bi-Weekly', 'Monthly']

results_by_freq = {}

print("Analyzing rebalancing frequency impact...\n")

for freq, name in zip(rebalance_frequencies, frequency_names):
    pnl, errors, costs = run_monte_carlo_hedging(
        n_simulations=500,
        S0=100, K=100, T=0.25, r=0.05, sigma=0.20, mu=0.08,
        n_steps=63, rebalance_freq=freq, transaction_cost=0.001
    )
    
    results_by_freq[name] = {
        'pnl_mean': np.mean(pnl),
        'pnl_std': np.std(pnl),
        'pnl_5th': np.percentile(pnl, 5),
        'pnl_95th': np.percentile(pnl, 95),
        'avg_costs': np.mean(costs),
        'pnl_data': pnl
    }
    
    print(f"{name}:")
    print(f"  Mean P&L: ${results_by_freq[name]['pnl_mean']:.4f}")
    print(f"  Std Dev: ${results_by_freq[name]['pnl_std']:.4f}")
    print(f"  Avg Costs: ${results_by_freq[name]['avg_costs']:.4f}")
    print()

In [ ]:
# Visualize rebalancing frequency comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. P&L Distribution by Frequency
ax1 = axes[0]
positions = np.arange(len(frequency_names))
data = [results_by_freq[name]['pnl_data'] for name in frequency_names]
bp = ax1.boxplot(data, positions=positions, patch_artist=True)

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(frequency_names)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_xticks(positions)
ax1.set_xticklabels(frequency_names, rotation=15)
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('P&L ($)')
ax1.set_title('Hedging P&L by Rebalancing Frequency')
ax1.grid(True, alpha=0.3)

# 2. Risk vs Cost Trade-off
ax2 = axes[1]
std_devs = [results_by_freq[name]['pnl_std'] for name in frequency_names]
avg_costs = [results_by_freq[name]['avg_costs'] for name in frequency_names]

scatter = ax2.scatter(avg_costs, std_devs, c=range(len(frequency_names)), 
                      cmap='viridis', s=150, edgecolors='black')
for i, name in enumerate(frequency_names):
    ax2.annotate(name, (avg_costs[i], std_devs[i]), 
                 xytext=(10, 5), textcoords='offset points', fontsize=10)

ax2.set_xlabel('Average Transaction Costs ($)')
ax2.set_ylabel('P&L Standard Deviation ($)')
ax2.set_title('Risk vs. Cost Trade-off')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Volatility Mismatch Analysis (Model Risk)

In [ ]:
def analyze_vol_mismatch(hedge_vol, realized_vol, n_simulations=500):
    """
    Analyze hedging performance when hedge volatility differs from realized.
    """
    simulator = StockSimulator(
        S0=100, mu=0.08, sigma=realized_vol, T=0.25, 
        n_steps=63, n_paths=n_simulations
    )
    times, all_prices = simulator.simulate(seed=42)
    
    pnl_results = []
    
    for i in range(n_simulations):
        strategy = DeltaHedgingStrategy(
            S0=100, K=100, T=0.25, r=0.05, sigma=hedge_vol,
            option_type='call', n_options=-1, transaction_cost=0.001
        )
        results = strategy.run_hedging_simulation(all_prices[i], times, rebalance_freq=1)
        final_pnl = strategy.calculate_final_pnl(results)
        pnl_results.append(final_pnl['total_pnl'])
    
    return np.array(pnl_results)

# Test different volatility mismatches
realized_vol = 0.20
hedge_vols = [0.15, 0.18, 0.20, 0.22, 0.25]

vol_mismatch_results = {}

print(f"Realized Volatility: {realized_vol*100:.0f}%\n")
print("Analyzing volatility mismatch impact...\n")

for hvol in hedge_vols:
    pnl = analyze_vol_mismatch(hvol, realized_vol, n_simulations=500)
    vol_mismatch_results[hvol] = {
        'pnl_mean': np.mean(pnl),
        'pnl_std': np.std(pnl),
        'pnl_data': pnl
    }
    
    mismatch = (hvol - realized_vol) * 100
    print(f"Hedge Vol: {hvol*100:.0f}% (mismatch: {mismatch:+.0f}%)")
    print(f"  Mean P&L: ${np.mean(pnl):.4f}")
    print(f"  Std Dev: ${np.std(pnl):.4f}")
    print()

In [ ]:
# Visualize volatility mismatch impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. P&L Distribution by Hedge Volatility
ax1 = axes[0]
labels = [f'{v*100:.0f}%' for v in hedge_vols]
data = [vol_mismatch_results[v]['pnl_data'] for v in hedge_vols]

bp = ax1.boxplot(data, positions=range(len(hedge_vols)), patch_artist=True)
colors = ['red' if v < realized_vol else 'green' if v > realized_vol else 'blue' for v in hedge_vols]
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.5)

ax1.set_xticks(range(len(hedge_vols)))
ax1.set_xticklabels(labels)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.axvline(x=2, color='blue', linestyle=':', alpha=0.5, label='Correct Vol')
ax1.set_xlabel('Hedge Volatility')
ax1.set_ylabel('P&L ($)')
ax1.set_title(f'Hedging P&L by Implied Volatility (Realized: {realized_vol*100:.0f}%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Mean P&L vs Volatility Mismatch
ax2 = axes[1]
mismatches = [(v - realized_vol) * 100 for v in hedge_vols]
mean_pnls = [vol_mismatch_results[v]['pnl_mean'] for v in hedge_vols]
std_pnls = [vol_mismatch_results[v]['pnl_std'] for v in hedge_vols]

ax2.errorbar(mismatches, mean_pnls, yerr=std_pnls, fmt='o-', capsize=5, 
             color='navy', linewidth=2, markersize=10)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax2.axvline(x=0, color='green', linestyle='--', alpha=0.5)
ax2.set_xlabel('Volatility Mismatch (%)')
ax2.set_ylabel('Mean P&L ($)')
ax2.set_title('Mean P&L vs. Volatility Mismatch')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Underestimating volatility (hedge vol < realized) leads to losses,")
print("   while overestimating leads to profits on average (but at higher cost).")

## 9. Performance Metrics Dashboard

In [ ]:
class HedgingPerformanceMetrics:
    """
    Calculate comprehensive hedging performance metrics.
    """
    
    @staticmethod
    def calculate_metrics(pnl_array, transaction_costs, initial_premium):
        """
        Calculate hedging performance metrics.
        
        Parameters:
        -----------
        pnl_array : array - P&L from multiple simulations
        transaction_costs : array - Transaction costs from simulations
        initial_premium : float - Initial option premium received
        
        Returns:
        --------
        dict : Performance metrics
        """
        metrics = {}
        
        # Basic statistics
        metrics['mean_pnl'] = np.mean(pnl_array)
        metrics['std_pnl'] = np.std(pnl_array)
        metrics['min_pnl'] = np.min(pnl_array)
        metrics['max_pnl'] = np.max(pnl_array)
        metrics['median_pnl'] = np.median(pnl_array)
        
        # Percentiles
        metrics['pnl_1st'] = np.percentile(pnl_array, 1)
        metrics['pnl_5th'] = np.percentile(pnl_array, 5)
        metrics['pnl_95th'] = np.percentile(pnl_array, 95)
        metrics['pnl_99th'] = np.percentile(pnl_array, 99)
        
        # Risk metrics
        metrics['var_95'] = -np.percentile(pnl_array, 5)  # Value at Risk
        metrics['var_99'] = -np.percentile(pnl_array, 1)
        metrics['cvar_95'] = -np.mean(pnl_array[pnl_array <= np.percentile(pnl_array, 5)])  # Expected Shortfall
        
        # Efficiency metrics
        metrics['hedging_efficiency'] = 1 - (metrics['std_pnl'] / initial_premium)
        metrics['sharpe_ratio'] = metrics['mean_pnl'] / metrics['std_pnl'] if metrics['std_pnl'] > 0 else 0
        
        # Transaction cost analysis
        metrics['avg_transaction_costs'] = np.mean(transaction_costs)
        metrics['cost_as_pct_premium'] = metrics['avg_transaction_costs'] / initial_premium * 100
        
        # Win rate
        metrics['win_rate'] = np.mean(pnl_array > 0) * 100
        metrics['profit_factor'] = (
            np.sum(pnl_array[pnl_array > 0]) / abs(np.sum(pnl_array[pnl_array < 0]))
            if np.sum(pnl_array[pnl_array < 0]) != 0 else np.inf
        )
        
        return metrics
    
    @staticmethod
    def print_metrics_report(metrics, title="Hedging Performance Report"):
        """
        Print formatted performance report.
        """
        print("\n" + "="*60)
        print(f" {title}")
        print("="*60)
        
        print("\n📈 P&L Statistics:")
        print(f"   Mean P&L:        ${metrics['mean_pnl']:>10.4f}")
        print(f"   Std Deviation:   ${metrics['std_pnl']:>10.4f}")
        print(f"   Median P&L:      ${metrics['median_pnl']:>10.4f}")
        print(f"   Min P&L:         ${metrics['min_pnl']:>10.4f}")
        print(f"   Max P&L:         ${metrics['max_pnl']:>10.4f}")
        
        print("\n📊 Percentiles:")
        print(f"   1st Percentile:  ${metrics['pnl_1st']:>10.4f}")
        print(f"   5th Percentile:  ${metrics['pnl_5th']:>10.4f}")
        print(f"   95th Percentile: ${metrics['pnl_95th']:>10.4f}")
        print(f"   99th Percentile: ${metrics['pnl_99th']:>10.4f}")
        
        print("\n⚠️ Risk Metrics:")
        print(f"   VaR (95%):       ${metrics['var_95']:>10.4f}")
        print(f"   VaR (99%):       ${metrics['var_99']:>10.4f}")
        print(f"   CVaR (95%):      ${metrics['cvar_95']:>10.4f}")
        
        print("\n💰 Efficiency Metrics:")
        print(f"   Hedging Efficiency: {metrics['hedging_efficiency']*100:>8.2f}%")
        print(f"   Sharpe Ratio:       {metrics['sharpe_ratio']:>10.4f}")
        print(f"   Win Rate:           {metrics['win_rate']:>8.2f}%")
        print(f"   Profit Factor:      {metrics['profit_factor']:>10.4f}")
        
        print("\n💸 Transaction Costs:")
        print(f"   Average Costs:      ${metrics['avg_transaction_costs']:>10.4f}")
        print(f"   Costs (% Premium):  {metrics['cost_as_pct_premium']:>8.2f}%")
        
        print("\n" + "="*60)

# Calculate metrics for daily rebalancing
initial_premium = BlackScholesModel(100, 100, 0.25, 0.05, 0.20, 'call').price()

metrics = HedgingPerformanceMetrics.calculate_metrics(
    pnl_daily, costs_daily, initial_premium
)

HedgingPerformanceMetrics.print_metrics_report(metrics, "Daily Rebalancing Performance")

## 10. Comparison: Different Market Conditions

In [ ]:
# Compare hedging under different market conditions
scenarios = {
    'Low Vol (15%)': {'sigma': 0.15, 'mu': 0.05},
    'Normal Vol (20%)': {'sigma': 0.20, 'mu': 0.08},
    'High Vol (30%)': {'sigma': 0.30, 'mu': 0.10},
    'Bull Market': {'sigma': 0.20, 'mu': 0.20},
    'Bear Market': {'sigma': 0.25, 'mu': -0.15},
}

scenario_results = {}
n_sims = 500

print("Analyzing different market scenarios...\n")

for name, params in scenarios.items():
    simulator = StockSimulator(
        S0=100, mu=params['mu'], sigma=params['sigma'], 
        T=0.25, n_steps=63, n_paths=n_sims
    )
    times, all_prices = simulator.simulate(seed=42)
    
    pnl_results = []
    costs_results = []
    
    for i in range(n_sims):
        # Hedge using same implied vol (20%) - model risk!
        strategy = DeltaHedgingStrategy(
            S0=100, K=100, T=0.25, r=0.05, sigma=0.20,
            option_type='call', n_options=-1, transaction_cost=0.001
        )
        results = strategy.run_hedging_simulation(all_prices[i], times, rebalance_freq=1)
        final_pnl = strategy.calculate_final_pnl(results)
        pnl_results.append(final_pnl['total_pnl'])
        costs_results.append(final_pnl['cumulative_transaction_costs'])
    
    pnl_results = np.array(pnl_results)
    scenario_results[name] = {
        'pnl_data': pnl_results,
        'mean_pnl': np.mean(pnl_results),
        'std_pnl': np.std(pnl_results),
        'avg_costs': np.mean(costs_results)
    }
    
    print(f"{name}:")
    print(f"  Mean P&L: ${scenario_results[name]['mean_pnl']:.4f}")
    print(f"  Std P&L: ${scenario_results[name]['std_pnl']:.4f}")
    print()

In [ ]:
# Visualize scenario comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. P&L Distribution by Scenario
ax1 = axes[0]
scenario_names = list(scenarios.keys())
data = [scenario_results[name]['pnl_data'] for name in scenario_names]

bp = ax1.boxplot(data, positions=range(len(scenario_names)), patch_artist=True)
colors = plt.cm.Set3(np.linspace(0, 1, len(scenario_names)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax1.set_xticks(range(len(scenario_names)))
ax1.set_xticklabels(scenario_names, rotation=20, ha='right')
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('P&L ($)')
ax1.set_title('Hedging P&L by Market Scenario')
ax1.grid(True, alpha=0.3)

# 2. Summary Bar Chart
ax2 = axes[1]
x = np.arange(len(scenario_names))
width = 0.35

means = [scenario_results[name]['mean_pnl'] for name in scenario_names]
stds = [scenario_results[name]['std_pnl'] for name in scenario_names]

bars1 = ax2.bar(x - width/2, means, width, label='Mean P&L', color='steelblue')
bars2 = ax2.bar(x + width/2, stds, width, label='Std Dev', color='coral')

ax2.set_xticks(x)
ax2.set_xticklabels(scenario_names, rotation=20, ha='right')
ax2.set_ylabel('Value ($)')
ax2.set_title('Mean P&L and Risk by Scenario')
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 11. Summary and Key Takeaways

In [ ]:
# Create summary visualization
fig = plt.figure(figsize=(16, 10))

# Overall layout
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# 1. Rebalancing frequency impact
ax1 = fig.add_subplot(gs[0, 0])
freq_names = list(results_by_freq.keys())
means = [results_by_freq[f]['pnl_mean'] for f in freq_names]
stds = [results_by_freq[f]['pnl_std'] for f in freq_names]
ax1.errorbar(range(len(freq_names)), means, yerr=stds, fmt='o-', capsize=5, color='navy')
ax1.set_xticks(range(len(freq_names)))
ax1.set_xticklabels(['D', '2D', 'W', '2W', 'M'])
ax1.set_xlabel('Rebalancing Frequency')
ax1.set_ylabel('P&L ($)')
ax1.set_title('Impact of Rebalancing Frequency')
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.grid(True, alpha=0.3)

# 2. Volatility mismatch impact
ax2 = fig.add_subplot(gs[0, 1])
vol_labels = [f'{v*100:.0f}%' for v in vol_mismatch_results.keys()]
vol_means = [vol_mismatch_results[v]['pnl_mean'] for v in vol_mismatch_results.keys()]
colors = ['red' if v < 0.20 else 'green' if v > 0.20 else 'blue' for v in vol_mismatch_results.keys()]
ax2.bar(vol_labels, vol_means, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Hedge Volatility')
ax2.set_ylabel('Mean P&L ($)')
ax2.set_title('Volatility Mismatch Effect')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.grid(True, alpha=0.3, axis='y')

# 3. Market scenario comparison
ax3 = fig.add_subplot(gs[0, 2])
scenario_means = [scenario_results[s]['mean_pnl'] for s in scenarios.keys()]
ax3.barh(list(scenarios.keys()), scenario_means, color=plt.cm.RdYlGn(
    (np.array(scenario_means) - min(scenario_means)) / (max(scenario_means) - min(scenario_means) + 1e-10)
))
ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
ax3.set_xlabel('Mean P&L ($)')
ax3.set_title('Performance by Market Scenario')
ax3.grid(True, alpha=0.3, axis='x')

# 4. Cost vs Risk trade-off (larger)
ax4 = fig.add_subplot(gs[1, :2])
for name in results_by_freq.keys():
    ax4.scatter(results_by_freq[name]['avg_costs'], results_by_freq[name]['pnl_std'], 
                s=200, label=name, edgecolors='black')
ax4.set_xlabel('Transaction Costs ($)', fontsize=12)
ax4.set_ylabel('P&L Volatility ($)', fontsize=12)
ax4.set_title('Transaction Cost vs. Hedging Risk Trade-off', fontsize=14)
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)

# 5. Key metrics table
ax5 = fig.add_subplot(gs[1, 2])
ax5.axis('off')
table_data = [
    ['Metric', 'Value'],
    ['Mean P&L', f'${metrics["mean_pnl"]:.4f}'],
    ['Std Dev', f'${metrics["std_pnl"]:.4f}'],
    ['VaR (95%)', f'${metrics["var_95"]:.4f}'],
    ['CVaR (95%)', f'${metrics["cvar_95"]:.4f}'],
    ['Win Rate', f'{metrics["win_rate"]:.1f}%'],
    ['Hedge Efficiency', f'{metrics["hedging_efficiency"]*100:.1f}%'],
    ['Avg Cost', f'${metrics["avg_transaction_costs"]:.4f}']
]
table = ax5.table(cellText=table_data, loc='center', cellLoc='center',
                  colWidths=[0.5, 0.5])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.8)
ax5.set_title('Key Performance Metrics\n(Daily Rebalancing)', fontsize=12, pad=20)

plt.suptitle('Delta Hedging Strategy - Performance Summary', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 12. Key Learnings

### Delta Hedging Fundamentals
1. **Delta hedging** creates a risk-neutral portfolio by holding Δ shares against each option
2. **Rebalancing** is necessary as delta changes with price and time
3. **Transaction costs** create a trade-off between hedging accuracy and cost

### Performance Drivers
1. **Rebalancing frequency**: More frequent = better hedge, higher costs
2. **Volatility mismatch**: Using wrong vol introduces systematic P&L bias
3. **Gamma risk**: High gamma periods (near strike, near expiry) cause larger hedge errors

### Risk Management Insights
1. **VaR and CVaR** quantify tail risk from hedging errors
2. **Transaction costs** can exceed hedge P&L in volatile markets
3. **Model risk** from vol mismatch is a major concern

### Practical Considerations
1. Weekly rebalancing offers good cost-risk balance
2. Overhedging (using higher vol) provides cushion against volatility spikes
3. Deep hedging (ML approach) can optimize for transaction costs

---

## Exercises

1. **Implement gamma hedging**: Extend the strategy to hedge both delta and gamma using another option
2. **Stochastic volatility**: Test hedging performance under Heston model dynamics
3. **Optimal rebalancing**: Implement a threshold-based rebalancing strategy
4. **Deep hedging**: Use neural networks to learn optimal hedging policy
5. **Greeks surface**: Visualize how Greeks change with spot and time

---

**Next**: Week 17 continues with Deep Hedging using reinforcement learning!